In [ ]:
setwd('/import/hnsc/data/')

count = read.csv('./TCGA-HNSC.htseq_counts.tsv',sep = '\t',check.names = F)
gtf <- rtracklayer::import('./Homo_sapiens.GRCh38.112.gtf.gz')#转换为数据框
pheo = read.csv('./TCGA-HNSC.GDC_phenotype.csv') %>% dplyr::select(.,c('submitter_id.samples','perineural_invasion_present')) 
d = read.table('old.related.txt',header =F)

library(tidyr)
library(dplyr)
count <- count %>% 
  tidyr::separate(Ensembl_ID,into = c("Ensembl_ID"),sep="\\.") 


gtf <-as.data.frame(gtf)

colnames(count)[1]<-'gene_id'

a=dplyr::filter(gtf,type=="gene",gene_biotype=="protein_coding")
dim(a)

b=dplyr::select(a,c(gene_id,gene_name,gene_biotype))
b[1:4,]
c=dplyr::inner_join(b,count,by ="gene_id")
c[1:5,1:5]

tail(count,10)

library(dplyr)

d=dplyr::select(c,-gene_id,-gene_biotype)
mRNAdata=distinct(d,gene_name,.keep_all = T)

mRNAdata= mRNAdata[!duplicated(mRNAdata$gene_name),]

mRNAdata = na.omit(mRNAdata)

dim(mRNAdata)

rownames(mRNAdata)<-mRNAdata[,1]
mRNAdata<-mRNAdata[,-1]

mRNAdata<-2^mRNAdata -1

library(stringr)
tumor <- colnames(mRNAdata)[as.integer(substr(colnames(mRNAdata),14,15)) < 10]
normal <- colnames(mRNAdata)[as.integer(substr(colnames(mRNAdata),14,15)) >= 10]

tumor_sample <- mRNAdata[,tumor]
normal_sample <- mRNAdata[,normal]



rownames(pheo) = pheo[,1]

head(pheo,5)

PNI.yes = pheo %>% dplyr::filter(perineural_invasion_present == 'YES') %>% rownames() %>% intersect(.,tumor)

PNI.no = pheo %>% dplyr::filter(perineural_invasion_present == 'NO') %>% rownames() %>% intersect(.,tumor)

length(PNI.no)

length(PNI.yes)

tumor_sample <- mRNAdata[,PNI.yes]
normal_sample <- mRNAdata[,PNI.no]

exprSet_by_group <- cbind(tumor_sample,normal_sample)
group_list <- c(rep('PNI.yes',ncol(tumor_sample)),rep('PNI.no',ncol(normal_sample)))

group_list = factor(group_list)
design <- model.matrix(~0+group_list)
rownames(design) = colnames(data)
colnames(design) <- levels(group_list)

library(limma)
library(edgeR)

data = exprSet_by_group

DGElist <- DGEList( counts = data, group = group_list )
keep_gene <- rowSums( cpm(DGElist) > 1 ) >= 2 # 自定义
table(keep_gene)
DGElist <- DGElist[ keep_gene, , keep.lib.sizes = FALSE ]

DGElist <- calcNormFactors( DGElist )
v <- voom(DGElist, design, plot = TRUE, normalize = "quantile")
fit <- lmFit(v, design)
cont.matrix <- makeContrasts(contrasts = c('PNI.yes-PNI.no'), levels = design)

fit2 <- contrasts.fit(fit, cont.matrix)
fit2 <- eBayes(fit2)

nrDEG_limma_voom = topTable(fit2, coef = 'PNI.yes-PNI.no', n = Inf)
nrDEG_limma_voom = na.omit(nrDEG_limma_voom)
head(nrDEG_limma_voom)


padj = 0.05 # 自定义
foldChange= 0 # 自定义
nrDEG_limma_voom_signif = nrDEG_limma_voom[(nrDEG_limma_voom$adj.P.Val < padj & 
                          (nrDEG_limma_voom$logFC>foldChange | nrDEG_limma_voom$logFC<(-foldChange))),]
nrDEG_limma_voom_signif = nrDEG_limma_voom_signif[order(nrDEG_limma_voom_signif$logFC),]

dim(nrDEG_limma_voom_signif)



sample_list <- list(DEGs = diff1$gene, CRGs = homo[,2])

library(venn)
        library(VennDiagram)
        color=c( "#3C5488B2","#00A087B2", 
             "#F39B7FB2","#91D1C2B2", 
             "#8491B4B2", "#DC0000B2", 
             "#7E6148B2","yellow", 
             "darkolivegreen1", "lightskyblue", 
             "darkgreen", "deeppink", "khaki2", 
             "firebrick", "brown1", "darkorange1", 
             "cyan1", "royalblue4", "darksalmon", 
             "darkgoldenrod1", "darkseagreen", "darkorchid")

q <- venn.diagram(sample_list,filename = NULL,
             fill = color[1:(length(sample_list))],
             alpha = 0.50, cat.col = rep('black', 2), 
             col = 'black', cex = 1, fontfamily = 'serif', cat.cex = 1, cat.fontfamily = 'serif', margin = 0.2
                             )
    pdf(paste0('./',"venn.pdf"))
    grid.draw(q)
    dev.off()

grid.draw(q)